In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = '/content/drive/MyDrive/data/'
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [5]:
import csv

In [6]:
train_yogiyo = pd.read_csv(f'{DATA_PATH}yogiyo_reviews_jsi.csv')
train_playstore = pd.read_csv(f'{DATA_PATH}playstore_df.csv')

In [7]:
new_columns = ['review','owner_reply']

cols = ['고객리뷰','사장댓글']
yogiyo = train_yogiyo[cols]
yogiyo.columns=new_columns

cols = ['content','replyContent']
playstore = train_playstore[cols]
playstore.columns=new_columns

In [8]:
result_df = pd.concat([yogiyo,playstore], axis=0)
result_df
result_df['review'] = result_df['review'].str.replace("[^a-zA-Z가-힣0-9 .,!?\'\"]" , "",regex=True) #포함 안된것들은 다 지우기
result_df['owner_reply'] = result_df['owner_reply'].str.replace("[^a-zA-Z가-힣0-9 .,!?\'\"]" , "",regex=True)


shuffled_df = result_df.sample(frac=1, random_state=42)  # frac=1은 모든 행을 선택, random_state는 재현 가능한 랜덤 설정

sample_df = shuffled_df.iloc[:1000]


result_df.shape, shuffled_df.shape, sample_df.shape

((107615, 2), (107615, 2), (1000, 2))

In [9]:
shuffled_df

,review,owner_reply
87733,강남구 도곡동은 되는데 왜 개포동은 안될까요 도곡에서 개포동까지 걸어서 5분도 안되...,고객님 안녕하세요. 대한민국 1등배달어플 배달의민족입니다. 늘 저희 배달의민족을 아...
66422,배민 좋아요,sungmin oh 고객님저희도 고객님 좋아요.. 우리 배달의민족을 좋아해 주셔서 ...
4088,다 좋은데 먹고싶은 메뉴 검색할 땐 여러지점이 나오는데 카테고리에 들어가서 보면 그...,"고양이님,안녕하세요. 배달의민족입니다.서비스 이용 시 큰 만족을 드리지 못해 마음이..."
2888,또시킬게요,자주 주문해주서서 감사 합니당 고객님 항상 저희 봄봄의 메뉴에 만족하시는거 같아서 ...
69346,배달 어플 굿 너무 잘쓰고 있서요,"안녕하세요. 대한민국 1등 배달 앱, 배달의민족 입니다.배달의민족 서비스 이용에 만..."
...,...,...
50921,확실히 다른 배달 어플보다 최고인듯. 우린 확실히 배달의 민족인듯.,"JH Roh님,안녕하세요 대한민국 1등 배달 앱, 배달의민족 입니다.배달의민족 ..."
72855,지금처럼 수수료 최소화해서 소상공인들 살려주세요,"안녕하세요. 대한민국 1등 배달 앱, 배달의민족 입니다.말씀해주신 것과 같이 배달의..."
99729,주문하고 15분뒤에 리뷰쓰라고 해서 떠서 바로 쓰기눌럿는데 15분뒤에쓸수있다고 안된...,안녕하세요 예진님 배달의민족 인사드립니다. 배달의민족에 리뷰 작성을 해주시려고 했는...
860,항상 맛있게 잘 먹고 있어요,주문해 주셔서 감사합니다.늘 한결같은 마음과 맛으로 정성을 다 하겠습니다.맛있는 초...


In [10]:
shuffled_df.to_csv(f'{DATA_PATH}shuffled_df.csv', index = False)

In [11]:
text = playstore['review']
text

0         쿠폰ㅋㅋㅋㅋㅋㅋ 추가) 다른 분이 언급했듯 배민1 쿠폰과 일반 쿠폰 혼재에 따른 편...
1               이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것
2                               이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요
3         아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배...
4                                                     좋아요!!
                                ...                        
103645                             통신이 아예 안 되네요 4g는 빵빵 터지는데
103646    배달의 민족 초반때부터 사용했었는데 예전이 오히려 나은거 같아요 위치설정도 왔다리갔...
103647                           가끔 와파떠잇는데 와파잡아야댄다고안들어가저요ㅡㅡ
103648                           어쩌라는거야.. 서울에서 안산까지 배달해줌ㅡㅡ?
103649                                         진짜맛없음돈받고광고하나
Name: review, Length: 103650, dtype: object

# 전처리 (문장추출)


- 문장단위로 분리 (.,?,!) 이외에도 다양한 구분기호를 활용


In [12]:
import re

# 패턴을 정의합니다. 여러 가지 구분 기호와 줄바꿈 문자를 포함합니다.
sentence_pattern = re.compile(r'[^.!?]*[.!?]|[^.!?]+$') # $로 구분기호 없이 끝나는 경우 포함   # 구분기호 없는 경우 포함 문장 분리 버전. 이 버전 or 아래 간단한 버전 쓰면될 듯

google_reviews_sentence = []

for review in text:  # text는 문자열 리뷰들의 리스트입니다
    if isinstance(review, str):  # 리뷰가 문자열인지 확인합니다
        sentences = sentence_pattern.findall(review)
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        google_reviews_sentence.append(sentences)
    else:
        review = '리뷰없음'
        google_reviews_sentence.append(review) # 리뷰 없는 경우도 갯수, 순서 맞추기 위해 넣어줌
        print(review)



리뷰없음
리뷰없음
리뷰없음
리뷰없음


In [13]:
len(google_reviews_sentence), len(playstore['review'])

(103650, 103650)

In [14]:
google_reviews_sentence

[['쿠폰ㅋㅋㅋㅋㅋㅋ 추가) 다른 분이 언급했듯 배민1 쿠폰과 일반 쿠폰 혼재에 따른 편의성 및 사용불만과 동일합니다.'],
 ['이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것'],
 ['이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요'],
 ['아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배민1 이렇게만 있거, 저번달에는 15000원이상 사용가능했는데 이번엔 2만원이상.',
  '.',
  '.',
  '?',
  '쿠폰을 쓰라는건지말라는건지 진짜 역대최악의 배달플랫폼이네요.',
  '요기요 쓰러감 배민지움'],
 ['좋아요!', '!'],
 ['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 걸린다.',
  '라이더마다 배달건수가 얼마인지는 모르지만 1시간 이상이다.',
  '그걸 감안 할 수있으면 시켜라 하고요.',
  '배달비를 인질삼아 이런저런 시도를 해보는건 좋은데 왜 피해를 일하는분들이나 이용자들이 받아야하는질 모르겠네요.',
  '이렇게 화를 내면서까지 이용할 매리트가 있는지 모르겠어요.',
  '다른 배달 어플을 사용하던 포장주문하던 해야할거같네요.',
  '참 ㅎㅎ음식점측에서는 음식을 완성해둔 상태에서 배민측에서 배달이 늦게잡혀서 그렇다라고하는데 제발 돈에 미치지말고 예전처럼 일해주셨으면 하네요.'],
 ['응.'],
 ['갑자기 어플이 안 들어가지는데 무슨 이유죠'],
 ['헐 배민1쿠폰은 쓸곳이 없는데 왜자꾸 준다고 난리난리이며, 8월 등급혜택쿠폰 쓰려다가 안써져서 보니 배민1만 사용가능?',
  '?',
  '그래도 배민 위주로 써왔는데 이번에 진심으로 정내미 뚝뚝 떨어져서 굳이 여기 찾아와서 리뷰 남깁니다;;; 배민1 한두번 쓰고말 고객을 위해 기존 수많은 고

- ... / ??? / ?! 등 구분기호 분리 시에 두개 이상인 경우 문장으로 인식하는 것 제거 처리

In [15]:
filtered_reviews = []

for review in google_reviews_sentence:
    filtered_review = []
    for sentence in review:
        if sentence not in ['.', '?', '!']:
            filtered_review.append(sentence)
    filtered_reviews.append(filtered_review)

google_reviews_sentence = filtered_reviews


In [16]:
len(google_reviews_sentence), len(playstore['review'])

(103650, 103650)

In [17]:
google_reviews_sentence


[['쿠폰ㅋㅋㅋㅋㅋㅋ 추가) 다른 분이 언급했듯 배민1 쿠폰과 일반 쿠폰 혼재에 따른 편의성 및 사용불만과 동일합니다.'],
 ['이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것'],
 ['이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요'],
 ['아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배민1 이렇게만 있거, 저번달에는 15000원이상 사용가능했는데 이번엔 2만원이상.',
  '쿠폰을 쓰라는건지말라는건지 진짜 역대최악의 배달플랫폼이네요.',
  '요기요 쓰러감 배민지움'],
 ['좋아요!'],
 ['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 걸린다.',
  '라이더마다 배달건수가 얼마인지는 모르지만 1시간 이상이다.',
  '그걸 감안 할 수있으면 시켜라 하고요.',
  '배달비를 인질삼아 이런저런 시도를 해보는건 좋은데 왜 피해를 일하는분들이나 이용자들이 받아야하는질 모르겠네요.',
  '이렇게 화를 내면서까지 이용할 매리트가 있는지 모르겠어요.',
  '다른 배달 어플을 사용하던 포장주문하던 해야할거같네요.',
  '참 ㅎㅎ음식점측에서는 음식을 완성해둔 상태에서 배민측에서 배달이 늦게잡혀서 그렇다라고하는데 제발 돈에 미치지말고 예전처럼 일해주셨으면 하네요.'],
 ['응.'],
 ['갑자기 어플이 안 들어가지는데 무슨 이유죠'],
 ['헐 배민1쿠폰은 쓸곳이 없는데 왜자꾸 준다고 난리난리이며, 8월 등급혜택쿠폰 쓰려다가 안써져서 보니 배민1만 사용가능?',
  '그래도 배민 위주로 써왔는데 이번에 진심으로 정내미 뚝뚝 떨어져서 굳이 여기 찾아와서 리뷰 남깁니다;;; 배민1 한두번 쓰고말 고객을 위해 기존 수많은 고객들 손절 마케팅 신박하네요ㅋ'],
 ['처음설치해서쓰려는데

In [18]:
text = shuffled_df['owner_reply']
text

87733    고객님 안녕하세요. 대한민국 1등배달어플 배달의민족입니다. 늘 저희 배달의민족을 아...
66422    sungmin oh 고객님저희도 고객님 좋아요.. 우리 배달의민족을 좋아해 주셔서 ...
4088     고양이님,안녕하세요. 배달의민족입니다.서비스 이용 시 큰 만족을 드리지 못해 마음이...
2888     자주 주문해주서서 감사 합니당 고객님 항상 저희 봄봄의 메뉴에 만족하시는거 같아서 ...
69346    안녕하세요. 대한민국 1등 배달 앱, 배달의민족 입니다.배달의민족 서비스 이용에 만...
                               ...                        
50921    JH Roh님,안녕하세요   대한민국 1등 배달 앱, 배달의민족 입니다.배달의민족 ...
72855    안녕하세요. 대한민국 1등 배달 앱, 배달의민족 입니다.말씀해주신 것과 같이 배달의...
99729    안녕하세요 예진님 배달의민족 인사드립니다. 배달의민족에 리뷰 작성을 해주시려고 했는...
860      주문해 주셔서 감사합니다.늘 한결같은 마음과 맛으로 정성을 다 하겠습니다.맛있는 초...
11830    미친치킨사줘?님,안녕하세요. 배달의민족입니다.앱 서비스 이용 시 아쉬우셨을 고객님 ...
Name: owner_reply, Length: 107615, dtype: object

In [19]:
sentence_endings = ['.', '?', '!']  # 간단한 문장 분리 버전.
total_reviews_sentence = []

for review in text: #위 text (모든 답변)
    sentences = []
    current_sentence = ""
    for char in review:
        current_sentence += char
        if char in sentence_endings:
            sentences.append(current_sentence.strip())
            current_sentence = ""
    total_reviews_sentence.append(sentences)

In [20]:
total_reviews_sentence

[['고객님 안녕하세요.',
  '대한민국 1등배달어플 배달의민족입니다.',
  '늘 저희 배달의민족을 아껴주셔서 정말 감사합니다.',
  '보다 다양한 혜택과 즐거움을 드릴 수 있도록 항상 노력하는 배달의민족이 되겠습니다.',
  '늦은 밤 출출할 때는 배달의민족!',
  '다이어트는 포샵으로!',
  '잊지 않으셨죠?',
  '오매불망 고객님만 생각하는 배달의민족을 기억해주세요!',
  '늘 고맙습니다.'],
 ['sungmin oh 고객님저희도 고객님 좋아요.',
  '.',
  '우리 배달의민족을 좋아해 주셔서 너무 기쁩니다.',
  '오늘도 힘!',
  '내서 더 든든한 서비스를 제공하기 위해 노력하는 배민 되겠습니다!',
  '!'],
 ['고양이님,안녕하세요.',
  '배달의민족입니다.',
  '서비스 이용 시 큰 만족을 드리지 못해 마음이 무겁습니다.',
  '배달의민족은 고객님께 다양한 가게를 소개해드리고 품질 좋은 서비스를 받으실 수 있도록 늘 고민하고 있습니다.',
  '가게의 대표 메뉴에 따라 알맞은 카테고리에 노출되고 있으니 이용에 참고 부탁드리며, 앞으로 더 많은 가게들을 이용하실 수 있도록 더욱 노력하겠습니다.',
  '지금처럼 꾸준한 애용 부탁드립니다.',
  '감사합니다.'],
 [],
 ['안녕하세요.',
  '대한민국 1등 배달 앱, 배달의민족 입니다.',
  '배달의민족 서비스 이용에 만족하신 것 같아서 정말 기쁩니다!',
  '앞으로도 고객님께서 알찬 식사 시간을 보내실 수 있도록, 끊임없이 고민하고 노력하겠습니다.',
  '지금처럼 배달의민족에 대한 많은 애정과 관심 부탁드립니다.',
  '감사합니다.'],
 ['이용성님,안녕하세요 대한민국 1등 배달 앱, 배달의민족 입니다.',
  '고객님 사랑합니다!',
  '.',
  '배달의민족 서비스가 마음에 드셨다는 표현인 것 같아 뿌듯함을 감출 수가 없습니다.',
  '앞으로도 배달의민족에 많은 애정과 관심 부탁드립니다.',
  '감사합니다.'],
 ['안녕하세요 배달의민족입

### 업체 관련 문장 (배달의 민족)

In [21]:
total_sentence = []

for review in total_reviews_sentence:
  for sentence in review :
    total_sentence.append(sentence)

In [22]:
len(total_sentence) ## 66만개의 문장

662670

In [23]:
company_sentence = []

for review in total_reviews_sentence:
    for sentence in review:
        if '배달의민족' in sentence:
            company_sentence.append(sentence)

# 추출된 문장들 출력
for sentence in company_sentence:
    print(sentence)

    break

대한민국 1등배달어플 배달의민족입니다.


In [24]:
len(company_sentence)

237968

In [25]:
pd.DataFrame(company_sentence).to_csv(f'{DATA_PATH}company_sentence.csv')

In [26]:
company_sentence

['대한민국 1등배달어플 배달의민족입니다.',
 '늘 저희 배달의민족을 아껴주셔서 정말 감사합니다.',
 '보다 다양한 혜택과 즐거움을 드릴 수 있도록 항상 노력하는 배달의민족이 되겠습니다.',
 '늦은 밤 출출할 때는 배달의민족!',
 '오매불망 고객님만 생각하는 배달의민족을 기억해주세요!',
 '우리 배달의민족을 좋아해 주셔서 너무 기쁩니다.',
 '배달의민족입니다.',
 '배달의민족은 고객님께 다양한 가게를 소개해드리고 품질 좋은 서비스를 받으실 수 있도록 늘 고민하고 있습니다.',
 '대한민국 1등 배달 앱, 배달의민족 입니다.',
 '배달의민족 서비스 이용에 만족하신 것 같아서 정말 기쁩니다!',
 '지금처럼 배달의민족에 대한 많은 애정과 관심 부탁드립니다.',
 '이용성님,안녕하세요 대한민국 1등 배달 앱, 배달의민족 입니다.',
 '배달의민족 서비스가 마음에 드셨다는 표현인 것 같아 뿌듯함을 감출 수가 없습니다.',
 '앞으로도 배달의민족에 많은 애정과 관심 부탁드립니다.',
 '안녕하세요 배달의민족입니다.',
 '반갑습니다 배달의민족 입니다.',
 '저희 배달의민족 많이 사랑해주시고 아껴주셔서 감사합니다.',
 '고객님의 소중한 의견 늘 감사드리며, 앞으로도 실망시켜 드리지 않도록 더욱더 노력하는 배달의민족 되겠습니다.',
 '또한, 고객님의 칭찬에 힘입어 더욱더 발전하는 배달의민족이 될테니 앞으로도 많은 애용 부탁드립니다.',
 '지금처럼 배달의민족에 대한 많은 애정과 관심 부탁드립니다.',
 '안녕하세요 문연진 고객님 배달의민족 입니다.',
 '배달의민족 리뷰문화에 대해 칭찬 해 주셔서 너무 감사합니다.',
 '고객님의 솔직한 리뷰, 평가, 후기등으로 인해 저희 배달의민족도 함께 쑥쑥 성장하고 있답니다.',
 '늘 이용해 주셔서 감사드리며  배달의민족으로 주문 하실 떄는 늘 행복한 식사시간 되세요!',
 '배달의민족입니다.',
 '앞으로도 배달의민족 앱 서비스에 만족하실 수 있도록 고객님의 입장에서 생각하고 고민하는 배달의민족이 되겠습니다.'

### 고객 관련 문장 (배달의 민족, 요기요)

- 이름 추출 예시

In [ ]:
!pip install kiwipiepy

In [27]:
from kiwipiepy import Kiwi

# Kiwi 객체 생성
kiwi = Kiwi()

# 가정한 문장
sentence = "김철수님과 박영희님은 학교에서 만났습니다."

# 문장을 형태소 분석하여 고유명사 추출
tokens = kiwi.analyze(sentence)

proper_nouns = []

# 형태소 분석 결과에서 고유명사 추출
for token in tokens[0][0]:
    if token[1] == 'NNP':  # 'NNP'는 고유명사 태그
        proper_nouns.append(token[0])

# 추출된 고유명사 출력
print("Extracted Proper Nouns:", proper_nouns)


Extracted Proper Nouns: ['김철수', '박영희']


In [ ]:
import tqdm

In [ ]:
from kiwipiepy import Kiwi

# Kiwi 객체 생성
kiwi = Kiwi()

kiwi.prepare()

person_sentences = []

# 문장들을 순회하며 고유명사(사람 이름)를 추출
for review in total_reviews_sentence:
    for sentence in review:
        tokens = kiwi.analyze(sentence)
        proper_nouns = [token[0] for token in tokens[0][0] if token[1] == 'NNP']

        # 일반 명사가 아닌 고유명사만 추출된 경우에만 추가
        if proper_nouns and not any(noun in proper_nouns for noun in ['안녕하세요', '대한민국', '라이더', '배민1', '할로윈', '즐겨찾기']):
            person_sentences.append(sentence)

# # 추출된 문장들 출력
# for sentence in person_sentences:
#     print(sentence)



In [ ]:
len(person_sentences)

In [ ]:
person_sentences

In [ ]:
pd.DataFrame(person_sentences).to_csv(f'{DATA_PATH}person_sentence.csv')

# 요식업 리뷰 추출

1. 지정 키워드로 - '맛', '음식'
2. 요식업 리뷰에서 추출한 형태소 키워드로 (요기요) - top5, top 10

In [28]:
google_reviews_sentence

[['쿠폰ㅋㅋㅋㅋㅋㅋ 추가) 다른 분이 언급했듯 배민1 쿠폰과 일반 쿠폰 혼재에 따른 편의성 및 사용불만과 동일합니다.'],
 ['이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것'],
 ['이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요'],
 ['아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배민1 이렇게만 있거, 저번달에는 15000원이상 사용가능했는데 이번엔 2만원이상.',
  '쿠폰을 쓰라는건지말라는건지 진짜 역대최악의 배달플랫폼이네요.',
  '요기요 쓰러감 배민지움'],
 ['좋아요!'],
 ['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 걸린다.',
  '라이더마다 배달건수가 얼마인지는 모르지만 1시간 이상이다.',
  '그걸 감안 할 수있으면 시켜라 하고요.',
  '배달비를 인질삼아 이런저런 시도를 해보는건 좋은데 왜 피해를 일하는분들이나 이용자들이 받아야하는질 모르겠네요.',
  '이렇게 화를 내면서까지 이용할 매리트가 있는지 모르겠어요.',
  '다른 배달 어플을 사용하던 포장주문하던 해야할거같네요.',
  '참 ㅎㅎ음식점측에서는 음식을 완성해둔 상태에서 배민측에서 배달이 늦게잡혀서 그렇다라고하는데 제발 돈에 미치지말고 예전처럼 일해주셨으면 하네요.'],
 ['응.'],
 ['갑자기 어플이 안 들어가지는데 무슨 이유죠'],
 ['헐 배민1쿠폰은 쓸곳이 없는데 왜자꾸 준다고 난리난리이며, 8월 등급혜택쿠폰 쓰려다가 안써져서 보니 배민1만 사용가능?',
  '그래도 배민 위주로 써왔는데 이번에 진심으로 정내미 뚝뚝 떨어져서 굳이 여기 찾아와서 리뷰 남깁니다;;; 배민1 한두번 쓰고말 고객을 위해 기존 수많은 고객들 손절 마케팅 신박하네요ㅋ'],
 ['처음설치해서쓰려는데

### 1. 지정 키워드 : 맛, 음식 지정

In [29]:
restaurant_keywords_1 = ["맛","음식"]  # 요식업 관련 키워드 리스트, '배달'은 뺌
food_related_reviews_1 = []
indices_1 = []

for idx, review in enumerate(google_reviews_sentence):
  for sentence in review :
    if any(keyword in sentence for keyword in restaurant_keywords_1):
      food_related_reviews_1.append(review)
      indices_1.append(idx)

In [30]:
indices_1

[5,
 13,
 20,
 20,
 28,
 28,
 44,
 46,
 47,
 59,
 63,
 67,
 101,
 109,
 115,
 115,
 129,
 135,
 140,
 155,
 155,
 157,
 158,
 158,
 158,
 160,
 165,
 166,
 173,
 174,
 179,
 179,
 181,
 187,
 189,
 189,
 189,
 193,
 194,
 195,
 195,
 196,
 198,
 208,
 212,
 228,
 237,
 252,
 286,
 290,
 290,
 295,
 300,
 304,
 316,
 319,
 325,
 325,
 325,
 339,
 339,
 349,
 353,
 353,
 353,
 357,
 361,
 362,
 363,
 376,
 376,
 384,
 388,
 392,
 394,
 403,
 405,
 413,
 413,
 413,
 428,
 429,
 441,
 442,
 454,
 454,
 454,
 468,
 483,
 484,
 512,
 513,
 514,
 515,
 522,
 525,
 533,
 534,
 535,
 535,
 538,
 542,
 554,
 588,
 590,
 593,
 596,
 616,
 616,
 617,
 618,
 629,
 630,
 638,
 647,
 669,
 674,
 702,
 708,
 716,
 726,
 732,
 732,
 740,
 746,
 747,
 747,
 747,
 758,
 761,
 771,
 778,
 780,
 781,
 782,
 783,
 785,
 787,
 794,
 794,
 797,
 804,
 808,
 812,
 838,
 849,
 902,
 915,
 915,
 915,
 922,
 932,
 943,
 956,
 972,
 975,
 977,
 987,
 990,
 994,
 994,
 1002,
 1002,
 1028,
 1030,
 1063,
 1063,
 1077

In [31]:
len(food_related_reviews_1)

11790

In [32]:
food_related_reviews_1

[['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 걸린다.',
  '라이더마다 배달건수가 얼마인지는 모르지만 1시간 이상이다.',
  '그걸 감안 할 수있으면 시켜라 하고요.',
  '배달비를 인질삼아 이런저런 시도를 해보는건 좋은데 왜 피해를 일하는분들이나 이용자들이 받아야하는질 모르겠네요.',
  '이렇게 화를 내면서까지 이용할 매리트가 있는지 모르겠어요.',
  '다른 배달 어플을 사용하던 포장주문하던 해야할거같네요.',
  '참 ㅎㅎ음식점측에서는 음식을 완성해둔 상태에서 배민측에서 배달이 늦게잡혀서 그렇다라고하는데 제발 돈에 미치지말고 예전처럼 일해주셨으면 하네요.'],
 ['아니진짜ㅡㅡ하 겁나 화딱지나서 요기요 쓰던가해야지 찜목록 편집기능을 만들어서 복수삭제가 가능하게해주든가ㅡㅡ 하나지우고나면 스크롤이상단으로 바껴져서 한참을 찾아내려야 겨우하나지우고ㅡㅡ지우기무서워서 업체 찜하기 누르겠냐고ㅡㅡ손목아파죽겠는데 이사하할때마다 딴동네업체들땜에 내가정작맛있게먹었던곳 찾기는힘들고 배달가능지역도아닌데 찜목록만 조잡해져서 정리하고 싶어도 개열받네 기본적인 기능도안되있고 노답'],
 ['몇년째 천생연분유지중.',
  '하루에 적으면2번 많으면4번씩 매일 주문중.',
  '✔이번 쿠폰사건이후로 ⭐혜택 좋은 요기요⭐로 갈아탑니다 ✔월간쿠폰에서 필요도없는 배민1쿠폰으로 바뀐거, 자영업자들 수수료 뜯어먹으려 한거죠뭐.',
  '같은 식당이라도 배민1으로 주문하려면 음식값이 8천원짜리가 1만원으로판매, 배달비는 무료~3천짜리가 기본4-5천입니다.',
  '쿠폰아~무의미 없습니다.',
  '✔알뜰배달도 마찬가지.',
  '알뜰 쿠폰남발해도 안써요.',
  '알뜰배달집 원래대로 시키면 배달비 무료집 많아요.',
  '쿠폰써봐야 천원할인.',
  '더 손해인데 누가쓰겠습

### 2. 요기요 요식업 키워드 : top 5, top10

In [ ]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602628 sha256=ee2148f8ac6b387b00bb6c2dab0c4b3258dc751b6c3e9e97ce298aa3b6de90e0
  Stored in directory: /root/.cache/pip/wheels/f3/55/41/ca474338ece1bc4314b01445f64ff002d71e19df45575a16e2
Successfully built kiwipiepy-model


In [ ]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [ ]:
from kiwipiepy.utils import Stopwords
stopwords =  Stopwords()

# # 불용어 추가
# additional_stopwords = ['안녕하세요',
#                         '불용어2', '불용어3']  # 추가할 불용어들을 리스트로 작성
# stopwords.add(additional_stopwords)

# # 변경된 불용어 리스트 확인
print(stopwords.stopwords)

In [ ]:
text = yogiyo['review']

restaurant_keywords_2 = []

for idx,text in enumerate(text):
    result = kiwi.tokenize(text,stopwords=stopwords)
    for token in result :
      if token.tag in ['VA','XR','NNG']: # 명사, 어근, 형용사
        restaurant_keywords_2.append(token.form)

In [ ]:
from collections import Counter

# restaurant_keywords_2는 키워드 리스트라고 가정합니다
# restaurant_keywords_2 = ['pizza', 'burger', 'sushi', 'pizza', 'sushi', 'ramen', 'burger', 'sushi']

# Counter를 사용하여 키워드 빈도를 계산합니다
keyword_frequencies = Counter(restaurant_keywords_2)

# 빈도순으로 키워드를 정렬합니다
sorted_keywords = sorted(keyword_frequencies.items(), key=lambda x: x[1], reverse=True)

print(sorted_keywords)


In [ ]:

# 튜플이 아닌 원소만 담긴 리스트 생성
keyword_list = [keyword for keyword, _ in sorted_keywords]
print(keyword_list)


In [ ]:
sorted_keywords[:50]

In [ ]:
restaurant_top5 = keyword_list[:5]
restaurant_top10 = keyword_list[:10]
restaurant_top50 = keyword_list[:50]
restaurant_top100 = keyword_list[:100]

restaurant_top5

In [ ]:
restaurant_top10

- top 5 키워드 : 55662

In [ ]:
restaurant_top5
food_related_reviews_2_top5 = []
indices_2_top5 = []

for idx, review in enumerate(google_reviews_sentence):
  for sentence in review :
    if any(keyword in sentence for keyword in restaurant_top5):
      food_related_reviews_2_top5.append(review)
      indices_2_top5.append(idx)

In [ ]:
len(food_related_reviews_2_top5)

In [ ]:
food_related_reviews_2_top5

[['이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것'],
 ['이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요'],
 ['아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배민1 이렇게만 있거, 저번달에는 15000원이상 사용가능했는데 이번엔 2만원이상.',
  '쿠폰을 쓰라는건지말라는건지 진짜 역대최악의 배달플랫폼이네요.',
  '요기요 쓰러감 배민지움'],
 ['아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배민1 이렇게만 있거, 저번달에는 15000원이상 사용가능했는데 이번엔 2만원이상.',
  '쿠폰을 쓰라는건지말라는건지 진짜 역대최악의 배달플랫폼이네요.',
  '요기요 쓰러감 배민지움'],
 ['좋아요!'],
 ['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 걸린다.',
  '라이더마다 배달건수가 얼마인지는 모르지만 1시간 이상이다.',
  '그걸 감안 할 수있으면 시켜라 하고요.',
  '배달비를 인질삼아 이런저런 시도를 해보는건 좋은데 왜 피해를 일하는분들이나 이용자들이 받아야하는질 모르겠네요.',
  '이렇게 화를 내면서까지 이용할 매리트가 있는지 모르겠어요.',
  '다른 배달 어플을 사용하던 포장주문하던 해야할거같네요.',
  '참 ㅎㅎ음식점측에서는 음식을 완성해둔 상태에서 배민측에서 배달이 늦게잡혀서 그렇다라고하는데 제발 돈에 미치지말고 예전처럼 일해주셨으면 하네요.'],
 ['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 

- top 10 키워드 : 66459


In [ ]:
restaurant_top10
food_related_reviews_2_top10 = []
indices_2_top10 = []

for idx, review in enumerate(google_reviews_sentence):
  for sentence in review :
    if any(keyword in sentence for keyword in restaurant_top10):
      food_related_reviews_2_top10.append(review)
      indices_2_top10.append(idx)

In [ ]:
len(food_related_reviews_2_top10)

66459

In [ ]:
food_related_reviews_2_top10

[['이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것'],
 ['이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요'],
 ['아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배민1 이렇게만 있거, 저번달에는 15000원이상 사용가능했는데 이번엔 2만원이상.',
  '쿠폰을 쓰라는건지말라는건지 진짜 역대최악의 배달플랫폼이네요.',
  '요기요 쓰러감 배민지움'],
 ['아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배민1 이렇게만 있거, 저번달에는 15000원이상 사용가능했는데 이번엔 2만원이상.',
  '쿠폰을 쓰라는건지말라는건지 진짜 역대최악의 배달플랫폼이네요.',
  '요기요 쓰러감 배민지움'],
 ['좋아요!'],
 ['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 걸린다.',
  '라이더마다 배달건수가 얼마인지는 모르지만 1시간 이상이다.',
  '그걸 감안 할 수있으면 시켜라 하고요.',
  '배달비를 인질삼아 이런저런 시도를 해보는건 좋은데 왜 피해를 일하는분들이나 이용자들이 받아야하는질 모르겠네요.',
  '이렇게 화를 내면서까지 이용할 매리트가 있는지 모르겠어요.',
  '다른 배달 어플을 사용하던 포장주문하던 해야할거같네요.',
  '참 ㅎㅎ음식점측에서는 음식을 완성해둔 상태에서 배민측에서 배달이 늦게잡혀서 그렇다라고하는데 제발 돈에 미치지말고 예전처럼 일해주셨으면 하네요.'],
 ['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 

# 모델학습

### 데이터 **마련**

#### 1. 지정 키워드

In [ ]:
indices_1

[5,
 13,
 20,
 20,
 28,
 28,
 44,
 46,
 47,
 59,
 63,
 67,
 101,
 109,
 115,
 115,
 129,
 135,
 140,
 155,
 155,
 157,
 158,
 158,
 158,
 160,
 165,
 166,
 173,
 174,
 179,
 179,
 181,
 187,
 189,
 189,
 189,
 193,
 194,
 195,
 195,
 196,
 198,
 208,
 212,
 228,
 237,
 252,
 286,
 290,
 290,
 295,
 300,
 304,
 316,
 319,
 325,
 325,
 325,
 339,
 339,
 349,
 353,
 353,
 353,
 357,
 361,
 362,
 363,
 376,
 376,
 384,
 388,
 392,
 394,
 403,
 405,
 413,
 413,
 413,
 428,
 429,
 441,
 442,
 454,
 454,
 454,
 468,
 483,
 484,
 512,
 513,
 514,
 515,
 522,
 525,
 533,
 534,
 535,
 535,
 538,
 542,
 554,
 588,
 590,
 593,
 596,
 616,
 616,
 617,
 618,
 629,
 630,
 638,
 647,
 669,
 674,
 702,
 708,
 716,
 726,
 732,
 732,
 740,
 746,
 747,
 747,
 747,
 758,
 761,
 771,
 778,
 780,
 781,
 782,
 783,
 785,
 787,
 794,
 794,
 797,
 804,
 808,
 812,
 838,
 849,
 902,
 915,
 915,
 915,
 922,
 932,
 943,
 956,
 972,
 975,
 977,
 987,
 990,
 994,
 994,
 1002,
 1002,
 1028,
 1030,
 1063,
 1063,
 1077

In [ ]:
playstore

,review,owner_reply
0,쿠폰ㅋㅋㅋㅋㅋㅋ 추가) 다른 분이 언급했듯 배민1 쿠폰과 일반 쿠폰 혼재에 따른 편...,"o Dan님,\n\n안녕하세요. 배달의민족입니다.\n혹시 이용에 불편함이나, 만족스..."
1,이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것,"문상화님,\n\n안녕하세요. 배달의민족입니다.\n배달의민족 이용 후 좋다고 말씀해주..."
2,이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요,"풍기도리님,\n\n안녕하세요. 배달의민족입니다.\n할인 혜택이 부족하여 아쉬움을 드..."
3,"아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배...","롤린이배린이 (롤배그올리는채널)님,\n\n안녕하세요. 배달의민족입니다.\n혜택 부족..."
4,좋아요!!,"꼬냑소년님,\n\n안녕하세요. 배달의민족입니다.\n배달의민족 서비스 이용에 만족하신..."
...,...,...
103645,통신이 아예 안 되네요 4g는 빵빵 터지는데,안녕하세요. 배달의민족 입니다. 네트워크 문제로 불편을 드려 죄송합니다.\n\n배달...
103646,배달의 민족 초반때부터 사용했었는데 예전이 오히려 나은거 같아요 위치설정도 왔다리갔...,오랜 시간 보내주신 사랑을 먹고 무럭무럭 성장하고 있는 배달의민족 입니다. \n\n...
103647,가끔 와파떠잇는데 와파잡아야댄다고안들어가저요ㅡㅡ,와아파이 잡아야서 얼른 주문해야 되는데 계속 물어봐서 짜증나셨죠. 네트워크 오류가 ...
103648,어쩌라는거야.. 서울에서 안산까지 배달해줌ㅡㅡ?,안녕하세요 전국구 배달어플 배달의민족 입니다. 안산에서 서울의 업소정보를 보게 되셨...


In [ ]:
food_playstore_1 = playstore.iloc[indices_1]

In [ ]:
food_result_df_1 = pd.concat([yogiyo, food_playstore_1],axis=0 )

food_result_df_1.shape, yogiyo.shape, food_playstore_1.shape

((15755, 2), (3965, 2), (11790, 2))

In [ ]:
food_shuffled_df_1 = food_result_df_1.sample(frac=1, random_state=42)

food_shuffled_df_1

,review,owner_reply
3233,잘 먹었습니다 :),리뷰 감사합니다 ₍₍ (̨̡⸝⸝´꒳`⸝⸝)̧̢ ₎₎ 좋은 하루 보내세요^^
63910,음식이 맘에 들어요,Soyoun Sin님\n\n고객님 입맛에 맞는 맛있는 식사를 하셨다니 업소를 대신하...
2357,맛있게 잘먹었습니다 토핑 좋아요!,안녕하세요 프라텔로입니다!\n저희 프라텔로를 이용해주셔서 정말 감사드립니다!\n앞...
53837,배달음식 시킬때 편리해서 좋아요^^,"김신영님,\n\n안녕하세요. 배달의민족 입니다.\n말하기 귀찮을 때 어플로 간단히 ..."
38454,우리 동네 배달 음식점이 여기 다 있었네요..,"변선영님,\n\n안녕하세요. 대한민국 1등 배달 앱, 배달의민족 입니다.\n배달의민..."
...,...,...
9416,음식 주문 편리. 적립금?사용시 간혹 두어가게정도는 업체사정 거부가 될 때가 있는것...,"七沢みあ님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 칭찬과 소중한 리뷰 감사..."
92720,언제나 맛집 굿,이정현 고객님! 이렇게 저희 배달의민족을 칭찬해주셔서 정말 감사합니다. 고객님들의...
11225,ㅋㅋㅋ다망해가면서별점1점만주니깐리뷰도지워버리네요 ㅋㅋㅋㅋ1시간안에온다면서1시간지나서...,"남ᄏᄏᄏ님,\n\n안녕하세요. 배달의민족입니다.\n배달지연으로 불편하셨을 고객님 생..."
860,항상 맛있게 잘 먹고 있어요 👍,주문해 주셔서 감사합니다.\n\n늘 한결같은 마음과 맛으로 정성을 다 하겠습니다.\...


In [ ]:
food_shuffled_df_1.to_csv(f'{DATA_PATH}food_shuffled_df_1.csv', index = False)

#### 2. 추출 키워드

In [ ]:
food_related_reviews_2_top5

[['이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것'],
 ['이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요'],
 ['아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배민1 이렇게만 있거, 저번달에는 15000원이상 사용가능했는데 이번엔 2만원이상.',
  '쿠폰을 쓰라는건지말라는건지 진짜 역대최악의 배달플랫폼이네요.',
  '요기요 쓰러감 배민지움'],
 ['아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배민1 이렇게만 있거, 저번달에는 15000원이상 사용가능했는데 이번엔 2만원이상.',
  '쿠폰을 쓰라는건지말라는건지 진짜 역대최악의 배달플랫폼이네요.',
  '요기요 쓰러감 배민지움'],
 ['좋아요!'],
 ['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 걸린다.',
  '라이더마다 배달건수가 얼마인지는 모르지만 1시간 이상이다.',
  '그걸 감안 할 수있으면 시켜라 하고요.',
  '배달비를 인질삼아 이런저런 시도를 해보는건 좋은데 왜 피해를 일하는분들이나 이용자들이 받아야하는질 모르겠네요.',
  '이렇게 화를 내면서까지 이용할 매리트가 있는지 모르겠어요.',
  '다른 배달 어플을 사용하던 포장주문하던 해야할거같네요.',
  '참 ㅎㅎ음식점측에서는 음식을 완성해둔 상태에서 배민측에서 배달이 늦게잡혀서 그렇다라고하는데 제발 돈에 미치지말고 예전처럼 일해주셨으면 하네요.'],
 ['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 

In [ ]:
food_playstore_2_top5 = playstore.iloc[indices_2_top5]

In [ ]:
food_result_df_2_top5 = pd.concat([yogiyo, food_playstore_2_top5],axis=0 )

food_result_df_2_top5.shape, yogiyo.shape, food_playstore_2_top5.shape

((59627, 2), (3965, 2), (55662, 2))

In [ ]:


food_shuffled_df_2_top5 = food_result_df_2_top5.sample(frac=1, random_state=42)

food_shuffled_df_2_top5

,review,owner_reply
1736,탕슉이 약간 아쉽지만 그래도 만족스럽습니다\n짬뽕을 안시켰는데 국물을 주셔서 맛 보...,"안녕하세요\n항상 부족함 없는 식사 하실수 있도록\n초심 잃지않고 좋은재료, 정성담..."
79730,최악 돈먹고 배달 안해주고 폐기처분시킨 쓰래기들,안녕하세요. 배달의민족입니다. 업소의 주문누락 또는 지연으로 고객님께서 불편 겪으신...
9309,밑에 가게 안 나온다고 찡찡대는 인간아 스크롤만 뒤지게 내릴 생각을 하지 말고 검색...,"목동윤형빈님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 따뜻한 성원에 감사드립..."
97541,아기낳은지 2달밖에 안되서 외식은 꿈도 못꾸는데ㅋ 배민으로~ 맛난치킨.족발~ 엄청시...,"정윤 고객님~ 안녕하세요, 배달의민족 입니다. 새로운 식구와 함께 맛있는 식사 즐기..."
6790,영주 둘다 꼬치야타코야 정말 맛있어요... 최상의 맛...맛의 천국,"영주 4차선 장작불국밥님,\n\n안녕하세요. 배달의민족입니다.\n입맛에 맞는 식사를..."
...,...,...
96722,"바로결제도 잘 돼고, 배달도 빠르고, 포인트는 쥐 꼬리 만큼 주지만ㅋㅋ깨알같이 모아...",전유리나 고객님! 안녕하세요. 오늘도 고객님의 칭찬에 힘이나는 배달의민족 입니다. ...
70504,좋아요ㅎㅎ,안녕하세요. 배달의민족 입니다.\n고객님께서 좋다고 말씀해주시니 너무나 기쁩니다!^...
860,항상 맛있게 잘 먹고 있어요 👍,주문해 주셔서 감사합니다.\n\n늘 한결같은 마음과 맛으로 정성을 다 하겠습니다.\...
25352,좋아여,"woong jo님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 ""좋아요!!"" 한..."


In [ ]:
food_shuffled_df_2_top5.to_csv(f'{DATA_PATH}food_shuffled_df_2_top5.csv', index = False)

In [ ]:
food_related_reviews_2_top10

[['이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것'],
 ['이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요'],
 ['아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배민1 이렇게만 있거, 저번달에는 15000원이상 사용가능했는데 이번엔 2만원이상.',
  '쿠폰을 쓰라는건지말라는건지 진짜 역대최악의 배달플랫폼이네요.',
  '요기요 쓰러감 배민지움'],
 ['아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배민1 이렇게만 있거, 저번달에는 15000원이상 사용가능했는데 이번엔 2만원이상.',
  '쿠폰을 쓰라는건지말라는건지 진짜 역대최악의 배달플랫폼이네요.',
  '요기요 쓰러감 배민지움'],
 ['좋아요!'],
 ['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 걸린다.',
  '라이더마다 배달건수가 얼마인지는 모르지만 1시간 이상이다.',
  '그걸 감안 할 수있으면 시켜라 하고요.',
  '배달비를 인질삼아 이런저런 시도를 해보는건 좋은데 왜 피해를 일하는분들이나 이용자들이 받아야하는질 모르겠네요.',
  '이렇게 화를 내면서까지 이용할 매리트가 있는지 모르겠어요.',
  '다른 배달 어플을 사용하던 포장주문하던 해야할거같네요.',
  '참 ㅎㅎ음식점측에서는 음식을 완성해둔 상태에서 배민측에서 배달이 늦게잡혀서 그렇다라고하는데 제발 돈에 미치지말고 예전처럼 일해주셨으면 하네요.'],
 ['배달비가 너무 올라서 최대한 아껴보고자 배민1로 알뜰배달로 주문하거나 할인쿠폰들을 최대한 사용해서 주문하고있어요.',
  '알뜰배달문구에 다른 이용자들이 알수있도록 기재 좀 해주세요.',
  '알뜰배달은 라이더가 수십곳의 집들을 들렸다가오니깐 배달시간이 엄청 

In [ ]:
food_playstore_2_top10 = playstore.iloc[indices_2_top10]

In [ ]:
food_result_df_2_top10 = pd.concat([yogiyo, food_playstore_2_top10],axis=0 )

food_result_df_2_top10.shape, yogiyo.shape, food_playstore_2_top10.shape

((70424, 2), (3965, 2), (66459, 2))

In [ ]:


food_shuffled_df_2_top10 = food_result_df_2_top10.sample(frac=1, random_state=42)

food_shuffled_df_2_top10

,review,owner_reply
93360,이벤트나 혜택도 다양해서 좋고 무엇보다 주문했을때 업소에 연락이 안될경우 빠른처리와...,고객님! 안녕하세요 : ) 남녀노소 이용할 수 있는 배달앱! 배달의민족 입니다. 저...
46729,좋아요,"고객님,\n\n안녕하세요. 배달의민족 입니다.\n배달의민족을 통해 즐거운 식사 시간..."
64975,좋습니다.,"고객님의 ""좋아요"" 한마디에 감동이 물밀듯이 밀려옵니다!ㅜ_ㅜ\n앞으로도 고객님들의..."
102237,처음가보는 장소에서 치킨이나 피자를 먹고싶을때 번거롭게 하나하나 검색할필요없이 배달...,안녕하세요. 언제 어디서나 든든한 배달의민족입니다. 배달의민족은 낯선 곳에서도 배달...
102850,매우실용적이고 편리해서좋아요^^,인녕하세요. 매우 실용적인 배달의민족입니다. 배달의민족은 편리한 배달음식 주문을 위...
...,...,...
55999,좋아요,"고객님,\n\n안녕하세요. 배달의민족 입니다.\n고객님의 ""좋아요~"" 한마디에 감동..."
2947,배달팁 좋습니다. 좋은데 배달팁이 6천원인거는 배보다 배꼽이 더 큰거 아닌가요,"Astar: :님,\n\n안녕하세요. 배달의민족입니다.\n이용 시 불편을 드린 것 ..."
84435,지방 작은동네라 그런지 바로결제되는곳이 많이없긴한대 편하게 주문할수 있어서 좋네요 ...,안녕하세요.장미정 고객님 배달의민족 입니다. 항상 저희 배달의민족의 고민은 어떻게하...
860,항상 맛있게 잘 먹고 있어요 👍,주문해 주셔서 감사합니다.\n\n늘 한결같은 마음과 맛으로 정성을 다 하겠습니다.\...


In [ ]:
food_shuffled_df_2_top10.to_csv(f'{DATA_PATH}food_shuffled_df_2_top10.csv', index = False)

### 모델, 토크나이저 정의 (KoGPT)

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, #해당 모델의 토크나이저
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

In [ ]:
# sample_df = shuffled_df.iloc[:3000]
# sample_df = food_shuffled_df_1.iloc[:3000]
# sample_df = food_shuffled_df_2_top5.iloc[:3000]
sample_df = food_shuffled_df_2_top10.iloc[:3000]

sample_df.shape


In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.question = df["review"].tolist()
        self.answer = df["owner_reply"].tolist()
    def __len__(self):
        return len(self.question)

    def __getitem__(self,idx):
        return "<q>" + self.question[idx] + "</s><a>" + self.answer[idx] + "</s>"

In [ ]:
def collate_fn(batch): #  배치 데이터를 전처리(형식변환) # 데이터 길이 맞추기 위해 collate(모으다, 조립하다)
    x = tokenizer(batch, return_tensors="pt",padding=True)
    return {"x":x}

# return_tensors="pt" 토큰화된 결과를 PyTorch의 텐서 형식으로 반환 (Tensorflow('tf'), Numpy형식 가능('np'))

In [ ]:
dt = ChatDataset(sample_df)
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

In [ ]:
model(**batch["x"]).logits.shape

In [ ]:
tokenizer.pad_token_id

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):
        x = batch["x"].to(device)
        pred = model(**x).logits
        n_class = pred.shape[-1]
        pred = pred[:,:-1]
        pred = pred.reshape(-1,n_class)

        tgt = x["input_ids"][:,1:]
        tgt = tgt.flatten()

        mask = tgt != 3
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred,tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [ ]:
batch_size = 2
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 10

### 사전학습모델 비교

- 하나의 모델 점수 보기

In [ ]:
reset_seeds(SEED)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=3e-5)

train_dt = ChatDataset(sample_df)
train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

loss_list = []
for _ in tqdm(range(epochs)):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    loss_list.append(train_loss)

    print(train_loss)



In [ ]:
df = pd.DataFrame(loss_list)
df.columns = ['food_shuffle_3000']

# new_df = pd.DataFrame(loss_list)
# new_df.columns = ['']

# df = pd.concat([df,new_df],axis=1)
# df

In [ ]:
df

In [ ]:
# model.save_pretrained(f"{DATA_PATH}kogpt2_chat1")
# model.save_pretrained(f"{DATA_PATH}kogpt2_chat2_food3000")
# model.save_pretrained(f"{DATA_PATH}kogpt2_chat3_food_top5_3000")
model.save_pretrained(f"{DATA_PATH}kogpt2_chat4_food_top10_3000")


- 모델별로 점수 담기 --> 다른 모델들 GPU 오류로 불가

In [ ]:
# import torch
# from transformers import AutoModelForCausalLM
# from tqdm import tqdm
# import pandas as pd

# # model_name = "skt/kogpt2-base-v2"  # 저장
# # # model_name = 'EleutherAI/polyglot-ko-5.8b' # 메모리 오류

# # # model_name = 'beomi/KoAlpaca-Polyglot-5.8B'
# # # model_name = 'nlpai-lab/kullm-polyglot-5.8b-v2'


# # 모델명과 학습 관련 파라미터들 설정
# model_name = 'skt/kogpt2-base-v2'  # 사전학습 모델의 이름
# epochs = 10
# batch_size = 2

# # 결과를 저장할 리스트 초기화
# epoch_scores = []

# # 시드 초기화 및 모델 생성
# reset_seeds(SEED)
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

# train_dt = ChatDataset(sample_df)
# train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# # 학습 루프 실행
# loss_list = []
# for epoch in tqdm(range(epochs)):
#     train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
#     loss_list.append(train_loss)

#     # 여기서 원하는 점수를 얻는 코드를 실행하고 epoch_scores 리스트에 추가
#     # 예를 들어, 테스트 데이터로 모델을 평가하여 점수를 계산하고 추가할 수 있습니다.
#     test_score = evaluate_model(model, test_dl)  # 적절한 함수로 평가 점수를 얻음
#     epoch_scores.append(test_score)

#     print(f"Epoch [{epoch + 1}/{epochs}] - Train Loss: {train_loss:.4f} - Test Score: {test_score:.4f}")

# # 결과를 데이터프레임으로 저장
# result_df = pd.DataFrame({'Epoch': range(1, epochs + 1), 'Test Score': epoch_scores})

# # 결과 출력
# print(result_df)


In [ ]:
# model.save_pretrained(f"{DATA_PATH}kogpt2_chat1") :





# model.save_pretrained(f"{DATA_PATH}EleutherAI_chat1")

# tokenizer.save_pretrained(f"{DATA_PATH}kogpt2_chat1_tokenizer")